PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2 MB 34 kB/s 
     |████████████████████████████████| 198 kB 21.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=917628c818ef3bae81cd2aaa7ae9a366344b53586b176a5a55fb3e26a91ebdd4
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Taipei Housing Price Prediction") \
    .getOrCreate()

In [3]:
spark

# 타이베이 주택 가격 예측 모델 만들기




데이터셋 설명

이번 문제는 대만 타이베이 시의 신단 지역에서 수집된 주택 거래 관련 정보를 바탕으로 주택 가격(정확히는 주택의 평당 가격)을 예측하는 Regression 모델을 만들어보는 것이다. 총 6개의 피쳐와 주택의 평당 가격에 해당하는 레이블 정보가 훈련 데이터로 제공된다. 레이블의 경우에는 주택의 최종 가격이 아니라 평당 가격이란 점을 다시 한번 강조한다.

각 컬럼에 대한 설명은 아래와 같으며 모든 필드는 X4를 제외하고는 실수 타입이다.

* X1: 주택 거래 날짜를 실수로 제공한다. 소수점 부분은 달을 나타낸다. 예를 들어 2013.250이라면 2013년 3월임을 나타낸다 (0.250 = 3/12)
* X2: 주택 나이 (년수)
* X3: 가장 가까운 지하철역까지의 거리 (미터)
* X4: 주택 근방 걸어갈 수 있는 거리내 편의점 수
* X5: 주택 위치의 위도 (latitude)
* X6: 주택 위치의 경도 (longitude)
* Y: 주택 평당 가격



In [4]:
!wget https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/Taipei_sindan_housing.csv

--2021-07-23 09:57:54--  https://grepp-reco-test.s3.ap-northeast-2.amazonaws.com/Taipei_sindan_housing.csv
Resolving grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)... 52.219.148.15
Connecting to grepp-reco-test.s3.ap-northeast-2.amazonaws.com (grepp-reco-test.s3.ap-northeast-2.amazonaws.com)|52.219.148.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20014 (20K) [text/csv]
Saving to: ‘Taipei_sindan_housing.csv’

Taipei_sindan_housi 100%[===================>]  19.54K   105KB/s    in 0.2s    

2021-07-23 09:57:55 (105 KB/s) - ‘Taipei_sindan_housing.csv’ saved [20014/20014]



In [5]:
!ls -tl

total 24
-rw-r--r-- 1 root root 20014 Jul 17 17:35 Taipei_sindan_housing.csv
drwxr-xr-x 1 root root  4096 Jul 16 13:20 sample_data


In [6]:
data = spark.read.csv('./Taipei_sindan_housing.csv', header=True, inferSchema=True)

In [7]:
data.printSchema()

root
 |-- X1: double (nullable = true)
 |-- X2: double (nullable = true)
 |-- X3: double (nullable = true)
 |-- X4: integer (nullable = true)
 |-- X5: double (nullable = true)
 |-- X6: double (nullable = true)
 |-- Y: double (nullable = true)



In [8]:
data.show()

+--------+----+--------+---+--------+---------+----+
|      X1|  X2|      X3| X4|      X5|       X6|   Y|
+--------+----+--------+---+--------+---------+----+
|2012.917|32.0|84.87882| 10|24.98298|121.54024|37.9|
|2012.917|19.5|306.5947|  9|24.98034|121.53951|42.2|
|2013.583|13.3|561.9845|  5|24.98746|121.54391|47.3|
|  2013.5|13.3|561.9845|  5|24.98746|121.54391|54.8|
|2012.833| 5.0|390.5684|  5|24.97937|121.54245|43.1|
|2012.667| 7.1| 2175.03|  3|24.96305|121.51254|32.1|
|2012.667|34.5|623.4731|  7|24.97933|121.53642|40.3|
|2013.417|20.3|287.6025|  6|24.98042|121.54228|46.7|
|  2013.5|31.7|5512.038|  1|24.95095|121.48458|18.8|
|2013.417|17.9| 1783.18|  3|24.96731|121.51486|22.1|
|2013.083|34.8|405.2134|  1|24.97349|121.53372|41.4|
|2013.333| 6.3|90.45606|  9|24.97433| 121.5431|58.1|
|2012.917|13.0|492.2313|  5|24.96515|121.53737|39.3|
|2012.667|20.4|2469.645|  4|24.96108|121.51046|23.8|
|  2013.5|13.2|1164.838|  4|24.99156|121.53406|34.3|
|2013.583|35.7|579.2083|  2| 24.9824|121.54619

In [9]:
data.select(['*']).describe().show()

+-------+------------------+------------------+------------------+------------------+--------------------+--------------------+------------------+
|summary|                X1|                X2|                X3|                X4|                  X5|                  X6|                 Y|
+-------+------------------+------------------+------------------+------------------+--------------------+--------------------+------------------+
|  count|               414|               414|               414|               414|                 414|                 414|               414|
|   mean|2013.1489710144933| 17.71256038647343|1083.8856889130436| 4.094202898550725|  24.969030072463745|  121.53336108695667| 37.98019323671498|
| stddev|0.2819672402629999|11.392484533242524| 1262.109595407851|2.9455618056636177|0.012410196590450208|0.015347183004592374|13.606487697735316|
|    min|          2012.667|               0.0|          23.38284|                 0|            24.93207|           1

In [10]:
from pyspark.sql.functions import isnan, when, count, col, isnull

data.select([count(when(isnull(c), c)).alias(c) for c in data.columns]).show()

+---+---+---+---+---+---+---+
| X1| X2| X3| X4| X5| X6|  Y|
+---+---+---+---+---+---+---+
|  0|  0|  0|  0|  0|  0|  0|
+---+---+---+---+---+---+---+




## Feature Vector 생성

In [11]:
from pyspark.ml.feature import VectorAssembler

feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
feature_columns

['X1', 'X2', 'X3', 'X4', 'X5', 'X6']

In [12]:
data_2 = assembler.transform(data)
data_2.show()

+--------+----+--------+---+--------+---------+----+--------------------+
|      X1|  X2|      X3| X4|      X5|       X6|   Y|            features|
+--------+----+--------+---+--------+---------+----+--------------------+
|2012.917|32.0|84.87882| 10|24.98298|121.54024|37.9|[2012.917,32.0,84...|
|2012.917|19.5|306.5947|  9|24.98034|121.53951|42.2|[2012.917,19.5,30...|
|2013.583|13.3|561.9845|  5|24.98746|121.54391|47.3|[2013.583,13.3,56...|
|  2013.5|13.3|561.9845|  5|24.98746|121.54391|54.8|[2013.5,13.3,561....|
|2012.833| 5.0|390.5684|  5|24.97937|121.54245|43.1|[2012.833,5.0,390...|
|2012.667| 7.1| 2175.03|  3|24.96305|121.51254|32.1|[2012.667,7.1,217...|
|2012.667|34.5|623.4731|  7|24.97933|121.53642|40.3|[2012.667,34.5,62...|
|2013.417|20.3|287.6025|  6|24.98042|121.54228|46.7|[2013.417,20.3,28...|
|  2013.5|31.7|5512.038|  1|24.95095|121.48458|18.8|[2013.5,31.7,5512...|
|2013.417|17.9| 1783.18|  3|24.96731|121.51486|22.1|[2013.417,17.9,17...|
|2013.083|34.8|405.2134|  1|24.97349|1

## Data Split

In [13]:
train, test = data_2.randomSplit([0.7, 0.3])

## 모델 생성

In [14]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor

### LinearRegression

In [15]:
linear = LinearRegression(featuresCol='features', labelCol='Y')
linear_model = linear.fit(train)

### RandomForestRegressor

In [16]:
rf = RandomForestRegressor(featuresCol='features', labelCol='Y')
rf_model = rf.fit(train)

### GBTRegressor

In [17]:
gbt = GBTRegressor(featuresCol='features', labelCol='Y')
gbt_model = gbt.fit(train)

## 성능 평가

In [18]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='Y', metricName='rmse')
evaluator

RegressionEvaluator_f014fdbe0fd4

In [19]:
linear_pred = linear_model.transform(test)
rf_pred = rf_model.transform(test)
gbt_pred = gbt_model.transform(test)

In [20]:
evaluator.evaluate(linear_pred), evaluator.evaluate(rf_pred), evaluator.evaluate(gbt_pred)

(8.988568117087084, 7.198784017933859, 8.44532447110716)

## MinMaxScaler 적용
  - ML Pipeline

In [21]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline

In [22]:
columns_to_scale = ['X1','X2','X3','X4','X5','X6']
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale]
scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_scale]

In [37]:
pipeline = Pipeline(stages = assemblers + scalers)
scalerModel = pipeline.fit(data)
scaledData = scalerModel.transform(data)

scaledData.show()

+--------+----+--------+---+--------+---------+----+----------+------+----------+------+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      X1|  X2|      X3| X4|      X5|       X6|   Y|    X1_vec|X2_vec|    X3_vec|X4_vec|    X5_vec|     X6_vec|           X1_scaled|           X2_scaled|           X3_scaled|           X4_scaled|           X5_scaled|           X6_scaled|
+--------+----+--------+---+--------+---------+----+----------+------+----------+------+----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|2012.917|32.0|84.87882| 10|24.98298|121.54024|37.9|[2012.917]|[32.0]|[84.87882]|[10.0]|[24.98298]|[121.54024]|[0.27292576419208...|[0.7305936073059361]|[0.00951267162646...|               [1.0]| [0.616941347552139]|[0.7193228380417942]|
|2012.917|19.5|306.5947|  9|24.98034|121.53951|4

In [38]:
scaledData = scaledData.select(['X1_scaled','X2_scaled','X3_scaled','X4_scaled','X5_scaled','X6_scaled','Y'])

In [39]:
feature_columns = scaledData.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
scaledData_2 = assembler.transform(scaledData)

scaledData_2.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+
|           X1_scaled|           X2_scaled|           X3_scaled|           X4_scaled|           X5_scaled|           X6_scaled|   Y|            features|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+
|[0.27292576419208...|[0.7305936073059361]|[0.00951267162646...|               [1.0]| [0.616941347552139]|[0.7193228380417942]|37.9|[0.27292576419208...|
|[0.27292576419208...|[0.44520547945205...|[0.04380939087238...|               [0.9]|[0.5849491032477332]|[0.7114513694199474]|42.2|[0.27292576419208...|
|[0.9999999999999999]|[0.30365296803652...|[0.0833150513098478]|               [0.5]| [0.671231216674745]|[0.7588958378261307]|47.3|[0.99999999999999...|
|[0.9093886462881348]|[0.30365296803652...|[0.0833150513098478]|            

In [26]:
scaledData_2.printSchema()

root
 |-- X1_scaled: vector (nullable = true)
 |-- X2_scaled: vector (nullable = true)
 |-- X3_scaled: vector (nullable = true)
 |-- X4_scaled: vector (nullable = true)
 |-- X5_scaled: vector (nullable = true)
 |-- X6_scaled: vector (nullable = true)
 |-- Y: double (nullable = true)
 |-- features: vector (nullable = true)



### 스케일링 전후 성능 비교

In [44]:
import pandas as pd

In [45]:
train_scaled, test_scaled = scaledData_2.randomSplit([0.7, 0.3])

linear_scaled_model = linear.fit(train_scaled)
rf_scaled_model = rf.fit(train_scaled)
gbt_scaled_model = gbt.fit(train_scaled)

linear_scaled_pred = linear_scaled_model.transform(test_scaled)
rf_scaled_pred = rf_scaled_model.transform(test_scaled)
gbt_scaled_pred = gbt_scaled_model.transform(test_scaled)

pd.DataFrame([[evaluator.evaluate(linear_pred), evaluator.evaluate(rf_pred), evaluator.evaluate(gbt_pred)], 
              [evaluator.evaluate(linear_scaled_pred), evaluator.evaluate(rf_scaled_pred), evaluator.evaluate(gbt_scaled_pred)]], 
             columns=['Linear', 'RandomForest', 'GBT'])

,Linear,RandomForest,GBT
0,8.988568,7.198784,8.445324
1,7.984950,6.443968,7.509495


- 전체적으로 RMSE 점수가 낮아졌다
  - ~사실 돌릴 때마다 들쭉날쭉 하는건 비밀~